In [ ]:
######## HELLO AGAIN CRISSY

# Cleaned up most of the code. Looks nicer, much more readable, with clear comments.
# Second model VERY similar to first. It makes sense since we haven't made any big changes.
# Today we make those big changes. Let's figure out the columns to drop based on p-values.
# At the end I added my log code with some notes.

# I see the light at the end of the tunnel. We'll get there!

In [ ]:
# Imports

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

df = pd.read_csv('kc_house_data.csv')

In [ ]:
# Examine

df.head()

In [ ]:
# Examine

df.info()

In [ ]:
# Move price column to end. Helps with formula creation (data.columns[:-1] is the easiest solution I could find to not including price~price in formula)

df = df[[c for c in df if c not in ['price']] + ['price']]

In [ ]:
# 'date' and 'sqft_basement' columns cause a problem with the model.summary() Don't know why. Further research needed.
# id column is clearly not part of the model

df = df.drop(['date','sqft_basement','id','yr_built'], axis=1)

#Would views or year renovated impact the price unless it was extremely dated? And, there are only two years on the year renovated category, which is limiting. We're thinking not likely, so we're going to remove those columns.  Also, we'd like to explore a way to fill the nan waterfront values but have decided to remove the column for now but may come back to. So we removed these columns
df=df.drop(columns=['yr_renovated', 'view','waterfront','sqft_lot15','sqft_lot15'])

In [ ]:

print(len(df[df.duplicated()]))
df = df.drop_duplicates()
print(df.isnull().sum().sum())
print(df.isnull().sum())

In [ ]:
# df['sqft_basement'] = pd.to_numeric(df['sqft_basement'], errors='coerce')
# print(df['sqft_basement'].isna().sum())
# df = df.dropna()
# print(df.isna().sum())

In [ ]:
data = df

In [ ]:
data.hist('price', bins=100)

In [ ]:
data['price'] = np.log(data['price'])
data.hist('price', bins=100)

In [ ]:
len(data)

In [ ]:
# Removing outliers!
# Step 1: Keep all homes with sale prices within 2 standard deviations of the mean.

std = data['price'].std()
mean = data['price'].mean()
data = data[(data['price'] >= (mean - (2 * std))) & (data['price'] <= (mean + (2 * std)))]

# Step 2: Split df into continuous and categorical values.

dfcontin = data[['sqft_living', 'sqft_lot', 'sqft_above']]
dfcat = data[['bedrooms', 'bathrooms', 'floors',
       'condition', 'grade', 'zipcode', 'lat',
       'long', 'price']]

# Step 3: Removing homes with outliers for all remaining continuous values.
# Obstacle was not wanting to run a for loop. A for loop would eliminate homes on each pass, leaving a smaller and smaller dataset with each pass, shrinking our data considerably.
# Found stats.zscore. Very useful. Wish I used that in the first place.
# 2 standard deviations eliminated too many homes. 2.5 seemed to be the sweet spot.

dfcontin = dfcontin[(np.abs(stats.zscore(data)) < 2.5).all(axis=1)]
data = dfcontin.merge(dfcat, left_index=True, right_index=True)
data.shape

In [ ]:
data.hist('price', bins=100)

In [ ]:
len(data)

In [ ]:
# Remaining homes have prices between $82,000 and $1,040,000.

print(data['price'].max())
print(data['price'].min())

In [ ]:
# Multiple regression model #1

formula = 'price ~ '+ '+'.join(data.columns[:-1])
model = ols(formula=formula, data=data).fit()
model.summary()

In [ ]:
sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

In [ ]:
# Train test split for linear regression.

data2 = data.copy()
data2['price'] = np.exp(data2['price'])
y = data2[['price']]
X = data2.drop(['price'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
# Compare RMSE.


train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error:', test_mse)

print('Train Root Mean Squared Error:', train_mse**0.5)
print('Test Root Mean Squared Error:', test_mse**0.5)

In [ ]:
# To help identify categorical columns

for col in data.columns:
    print(len(data[col].unique()), col)

In [ ]:
# From eyeballing it.

likelycategorical = ['bedrooms','bathrooms','floors','waterfront','view','condition','grade']

In [ ]:
# Inspect histograms for normality and better identify categoricals

data.hist(figsize=(18,20));

In [ ]:
# Scatter plots to visualize categorical values

fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(16,15), sharey=True)

for ax, column in zip(axes.flatten(), data.columns):
    ax.scatter(data[column], data['price']/100_000, label=column, alpha=.1)
    ax.set_title(f'Price vs {column}')
    ax.set_xlabel(column)  
fig.tight_layout()

In [ ]:
#sqft_living appears to be normally distributed.  And it looks promising with just a few outliers that we can clean up.

In [ ]:
# Heatmap to visualize correlation

plt.figure(figsize=(20,20))
df_corr = data.corr()
ax = sns.heatmap(df_corr, annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 1, top -1)

In [ ]:
#sqft above and sqft liv are highly correlated.  Since the sqft living appears to fit the baseline model better with fewer outliers, we will select it as our feature for our next iteration.

In [ ]:
#drop column that was strongly correlated to another feature

data.drop(columns=['sqft_above'])

In [ ]:
# Create dummy variables for categorical values.

floo_dummies = pd.get_dummies(data['floors'],prefix='floo',drop_first=True)
bath_dummies = pd.get_dummies(data['bathrooms'], prefix='bath',drop_first=True)
cond_dummies = pd.get_dummies(data['condition'], prefix='cond',drop_first=True)
grad_dummies = pd.get_dummies(data['grade'], prefix='grad',drop_first=True)
bed_dummies = pd.get_dummies(data['bedrooms'], prefix='bed',drop_first=True)

In [ ]:
# Concatenate. Behold our labor!
# Make a second DataFrame, bedxbath, preserving bed and bath columns. To be used later.

bedxbath = pd.concat([data, cond_dummies, grad_dummies ,floo_dummies], axis=1)
data = pd.concat([data,bed_dummies,bath_dummies, cond_dummies, grad_dummies, floo_dummies],axis=1)

In [ ]:
# Drop original columns to make room for dummies.

data.drop(['floors','bedrooms','bathrooms', 'condition', 'grade'],axis=1, inplace=True)

bedxbath.drop(['floors','condition','grade'],axis=1, inplace=True)

In [ ]:
# Once again, send price column to the end.

data = data[[c for c in data if c not in ['price']] + ['price']]

In [ ]:
# Discovered statsmodels is particular about symbols in column names. The '.' in our dummy variables such as 'bed_1.5' threw an error in our ols model.
# Solution: Create dictionary of {old_names : new_names} to use to rename columns.

cleankeys = list(data.columns)
cleanvalues = []
for c in data.columns:
    c = c.replace('.','_')
    cleanvalues.append(c)
    data
cleancols = dict(zip(cleankeys, cleanvalues))
data.rename(columns=cleancols, inplace=True)

In [ ]:
# ols works now with the new column names. 
# Inspect our second model.

formula = 'price ~ '+ ' + '.join(data.columns[:-1])
model = ols(formula=formula, data=data).fit()
model.summary()

In [ ]:
# Drop high p values

data.drop(['sqft_lot','sqft_above'], axis=1, inplace=True)

In [ ]:
sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

In [ ]:
# Make sure it looks right before we continue.

data.head()

In [ ]:
# Train test split for linear regression. Second round

data2 = data.copy()
y = data2[['price']]
X = data2.drop(['price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
# Compare our new RMSE again.

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error:', test_mse)

print('Train Root Mean Squared Error:', train_mse**0.5)
print('Test Root Mean Squared Error:', test_mse**0.5)

In [ ]:
# Very similar results. Time to start making more significant changes to see change.

In [ ]:
y = data['price']
cols = list(data.columns)
cols.remove('price')
pmax = 1
X = data[cols]
highpvalue = []
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
        highpvalue.append(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
print(highpvalue)

In [ ]:
# Still have a ton of variables at play. Inspired by Amber Yandow's lesson, decided to get rid of bed and bath dummy variables and process them in a different way.
# This is where earlier DataFrame named bedxbath comes into play.
# Creating new column, bedxbath, where # of bedrooms are multiplied by # of baths. Drop bed and bath columns.

bedxbath['bedxbath'] = bedxbath['bedrooms'] * bedxbath['bathrooms']
bedxbath.drop(['bedrooms','bathrooms','sqft_above','sqft_lot'], axis=1, inplace=True)

In [ ]:
bedxbath['bedxbath'].hist()
plt.show()
logtest = pd.DataFrame()
logtest['bedbathlog'] = np.log(bedxbath['bedxbath'])
logtest.hist()
plt.show()

In [ ]:
cleankeys = list(bedxbath.columns)
cleanvalues = []
for c in bedxbath.columns:
    c = c.replace('.','_')
    cleanvalues.append(c)
    data
cleancols = dict(zip(cleankeys, cleanvalues))
bedxbath.rename(columns=cleancols, inplace=True)

bedxbath = bedxbath[[c for c in bedxbath if c not in ['price']] + ['price']]

formula = 'price ~ '+ '+'.join(bedxbath.columns[:-1])
model = ols(formula=formula, data=bedxbath).fit()
model.summary()

In [ ]:
sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

In [ ]:
bedxbath.drop(['bedxbath'], axis=1, inplace=True)
bedxbath['bedbathlog'] = logtest['bedbathlog']

cleankeys = list(bedxbath.columns)
cleanvalues = []
for c in bedxbath.columns:
    c = c.replace('.','_')
    cleanvalues.append(c)
    data
cleancols = dict(zip(cleankeys, cleanvalues))
bedxbath.rename(columns=cleancols, inplace=True)

bedxbath = bedxbath[[c for c in bedxbath if c not in ['price']] + ['price']]

formula = 'price ~ '+ '+'.join(bedxbath.columns[:-1])
model = ols(formula=formula, data=bedxbath).fit()
model.summary()

In [ ]:
high_corr = ((abs(bedxbath.corr())> .8).sum()>1)
print(high_corr)
bedxbath.drop(['cond_3','cond_4'], axis=1, inplace=True)

In [ ]:
formula = 'price ~ '+ '+'.join(bedxbath.columns[:-1])
model = ols(formula=formula, data=bedxbath).fit()
model.summary()

In [ ]:
for column in bedxbath.columns:
    formula= f'price ~ + {column}'
    model = ols(formula=formula, data=bedxbath).fit()
    print(column)
    sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

In [ ]:
standardized = pd.DataFrame()
def scale(col):
    return(bedxbath[col] - bedxbath[col].mean()) / bedxbath[col].std()
price = pd.DataFrame()
price['logprice'] = data['price']
bedxbath.drop('price',axis=1)
for col in bedxbath.columns:
    standardized[col] = scale(col)
standardized = pd.concat([standardized, price], axis=1)

In [ ]:
standardized.drop('price', axis=1, inplace=True)

In [ ]:
standardized

In [ ]:
formula = 'logprice ~ '+ '+'.join(standardized.columns[:-1])
model = ols(formula=formula, data=standardized).fit()
model.summary()

In [ ]:
standardized.drop('floo_2_0', axis=1, inplace=True)
formula = 'logprice ~ '+ '+'.join(standardized.columns[:-1])
model = ols(formula=formula, data=standardized).fit()
model.summary()

In [ ]:
# Train test split for linear regression.

standardized2 = standardized.copy()
standardized2['logprice'] = np.exp(standardized2['logprice'])
y = standardized2[['logprice']]
X = standardized2.drop(['logprice'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
# Compare RMSE.

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error:', test_mse)

print('Train Root Mean Squared Error:', train_mse**0.5)
print('Test Root Mean Squared Error:', test_mse**0.5)

In [ ]:
################### THIS IS HERE TO WORK WITH TOMORROW
# Logs for some variables.

data_log = pd.DataFrame([])
data_log['log_living'] = np.log(data['sqft_living'])

logs = ['sqft_living']

data_log.hist(figsize= [6,6])
data[logs].hist(figsize= [6,6]);
plt.show()

In [ ]:
# For comparison, same variables without the log.
# Large improvement in 'sqft_lot' and 'sqft_lot15'.
# Medium improvement in 'sqft_above'


data[logs].hist(figsize= [6,6]);

In [ ]:
testlogs = pd.concat([data_log, data[logs], data['price']], axis=1)

formula = 'price ~ '+ '+'.join(testlogs.columns[:-1])
model = ols(formula=formula, data=testlogs).fit()
print(model.summary())
print(testlogs.columns[:-1], model.pvalues.values)

In [ ]:
# #Hi Tim!  I'm starting my convo here.  Also, I've added some comments throughout as Yish mention in the project documentation
# #to include questions and thoughts about our decisions throughout the notebook, #so trying to check off all the asks on the list
# #Here is a linear regression model we can use as our baseline.  It's not in the right spot, but we can figure that out after we
# #clean up the notebook a bit.  It needs to go after the train-test-split

# import statsmodels.api as sm
# import statsmodels.formula.api as smf
# import scipy.stats as stats
# import statsmodels.stats.api as sms

# results = []
# for idx, column in enumerate(data.columns):
#     print (f"SeattleHousingData - Regression Analysis and Diagnostics for price~{column}")
#     print ("--------------------------------------------------------------------------------------")
    
#     f = f'price~{column}'
#     model = smf.ols(formula=f, data=data).fit()
    
  
#     fig, axes = plt.subplots(figsize=(15,12))
#     fig = sm.graphics.plot_regress_exog(model, column, fig=fig)
#     fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)
#     fig.tight_layout()
#     plt.show()
    
#     results.append([column, model.rsquared, model.params[0], model.params[1], model.pvalues[1], sms.jarque_bera(model.resid)[0]])
#     input("Press Enter to continue...")

In [ ]:
model.pvalues.values

In [ ]:
data['zipcode'].value_counts()

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(16,15), sharey=True)
for ax, column in zip(axes.flatten(), df.columns):
    ax.scatter(df[column], df['price']/100_000, label=column, alpha=.1)
    ax.set_title(f'Price vs {column}')
    ax.set_xlabel(column)
fig.tight_layout()